# Import

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
os.environ['PYOPENGL_PLATFORM'] = 'egl'

In [ ]:
import torch
import trimesh
import open3d as o3d

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datareader import Ho3dReader

# Select mesh

In [ ]:
video_id = 'SM1'

In [ ]:
sam_dir = f'/home/simonep01/sam-3d-objects/meshes/{video_id}'
output = torch.load(f'{sam_dir}/output_data.pt', map_location='cuda:0')

# Load and pose mesh

In [ ]:
from pytorch3d.transforms import quaternion_to_matrix, Transform3d

In [ ]:
_R_ZUP_TO_YUP = np.array([[1, 0, 0], [0, 0, -1], [0, 1, 0]], dtype=np.float32)
_R_YUP_TO_ZUP = _R_ZUP_TO_YUP.T
my_rotation = np.array([[-1, 0, 0], [0, 1, 0], [0, 0, -1]], dtype=np.float32)

In [ ]:
def compose_transform(
    scale: torch.Tensor, rotation: torch.Tensor, translation: torch.Tensor
) -> Transform3d:
    """
    Args:
        scale: (..., 3) tensor of scale factors
        rotation: (..., 3, 3) tensor of rotation matrices
        translation: (..., 3) tensor of translation vectors
    """
    tfm = Transform3d(dtype=scale.dtype, device=scale.device)
    return tfm.scale(scale).rotate(rotation).translate(translation)

In [ ]:
mesh = output['glb'].copy()
mesh.export(f'{sam_dir}/initial_mesh.obj')

In [ ]:
# 1. Convert from Y-up (OBJ) to Z-up (transform space)
vertices = mesh.vertices.astype(np.float32) @ _R_YUP_TO_ZUP
vertices_tensor = torch.from_numpy(vertices).float().to(output["rotation"].device)

# 2. Apply transformation in Z-up space
R_l2c = quaternion_to_matrix(output['rotation'])
l2c_transform = compose_transform(
    scale=output['scale'],
    rotation=R_l2c,
    translation=output['translation'],
)
vertices_transformed = l2c_transform.transform_points(vertices_tensor.unsqueeze(0))
mesh.vertices = vertices_transformed.squeeze(0).cpu().numpy() @ my_rotation

In [ ]:
mesh.export(f'{sam_dir}/transformed_mesh.obj')

# Downsample mesh

In [ ]:
mesh = trimesh.load(f'{sam_dir}/transformed_mesh.obj')

In [ ]:
def downsample_mesh_best_quality(mesh, target_vertices):
    """
    Best quality mesh downsampling using direct quadric decimation.
    Preserves topology, sharp features, and minimizes geometric error.
    
    Args:
        mesh: input mesh
        target_vertices: Target number of vertices
    """
    print(f"Original: {len(mesh.vertices)} vertices, {len(mesh.faces)} faces")
    
    # Convert to Open3D mesh (preserve colors)
    mesh_o3d = o3d.geometry.TriangleMesh()
    mesh_o3d.vertices = o3d.utility.Vector3dVector(mesh.vertices)
    mesh_o3d.triangles = o3d.utility.Vector3iVector(mesh.faces)
    
    # Preserve vertex colors if available
    colors = mesh.visual.vertex_colors[:, :3] / 255.0
    mesh_o3d.vertex_colors = o3d.utility.Vector3dVector(colors)
    
    # Compute vertex normals for better quality
    mesh_o3d.compute_vertex_normals()
    
    # Quadric edge collapse decimation
    # Target triangles ≈ 2 * target_vertices for closed meshes
    target_triangles = target_vertices * 2
    
    mesh_simplified = mesh_o3d.simplify_quadric_decimation(
        target_number_of_triangles=target_triangles
    )
    
    print(f"Simplified: {len(mesh_simplified.vertices)} vertices, "
          f"{len(mesh_simplified.triangles)} faces")
    
    # Convert back to trimesh
    vertices_out = np.asarray(mesh_simplified.vertices)
    faces_out = np.asarray(mesh_simplified.triangles)
    
    colors_out = (np.asarray(mesh_simplified.vertex_colors) * 255).astype(np.uint8)
    mesh_out = trimesh.Trimesh(
        vertices=vertices_out,
        faces=faces_out,
        vertex_colors=colors_out
    )
    
    # Optional: Remove degenerate faces and fix normals
    mask = mesh_out.nondegenerate_faces()
    mesh_out.update_faces(mask)
    
    # Remove infinite/nan values
    mesh_out.remove_infinite_values()
    
    # Fix normals (use repair module)
    trimesh.repair.fix_normals(mesh_out)
    
    return mesh_out

In [ ]:
# Usage
mesh_simplified = downsample_mesh_best_quality(
    mesh=mesh,
    target_vertices=60000
)

In [ ]:
mesh_simplified.export(f'{sam_dir}/reduced_mesh.obj')

In [ ]:
mesh_simplified.show()